<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myProspects_Upside_Analysis_ATH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
def get_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

get_time()
gen_output = 0

Run date time (IST): 2023-08-03 09:13:46


In [2]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

In [3]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*10)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [4]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

def read_file(path):
 df = pd.read_csv(path)
 return df

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/myProspectsScrips.csv'
df_mypf = read_file(path_mypf)

#mypf = mypf[mypf['Nifty100'] == 1]

def get_stock_list(df_mypf):
  stock_n100 = df_mypf['Symbol'].unique()
  exclude = ['CADILAHC','MMTC']
  stock_n100 = df_mypf[~df_mypf['Symbol'].isin(exclude) ]['Symbol'].unique()
  stock_ids = stock_n100[:]
  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()
  return stock_ids

stock_ids = get_stock_list(df_mypf)

In [5]:
# star stocks & my stocks
star_stocks = list(df_mypf[df_mypf['StarStock'] == 1]['Stock'].values)
my_stocks = list(df_mypf[df_mypf['InPortfolio'] == 1]['Symbol'].values)
my_prospects = list(df_mypf[(df_mypf['LatestQtr'] == 1) & df_mypf['StarStock'] == 1]['Symbol'].values)

In [6]:
def print_distribution():

  # star stocks
  print('Overall - Star Stocks:')
  print(df_mypf['StarStock'].value_counts())
  print('-------')

  # v40/v40n stocks
  print('Overall - V40/V40N Stocks:')
  print(df_mypf['Criteria'].value_counts())
  print('-------')

  # latest quarter
  print('Overall - Latest Qtr:')
  print(df_mypf['LatestQtr'].value_counts())
  print('-------')

  # star + v40/v40n stocks
  print('Overall - Star + V40/V40N Stocks:')
  print(df_mypf[(df_mypf['Criteria'] == 'V40') | (df_mypf['Criteria'] == 'V40N')]['StarStock'].value_counts())
  print('-------')

  # stocks in portfolio
  print('Portfolio - Stocks:')
  print(df_mypf['InPortfolio'].value_counts())
  print('-------')

  # star stocks in portfolio
  print('Portfolio - Star Stocks:')
  print(df_mypf[(df_mypf['InPortfolio'] == 1)]['StarStock'].value_counts())
  print('-------')

  # latest quarter in portfolio
  print('Portfolio - Latest Quarter:')
  print(df_mypf[(df_mypf['InPortfolio'] == 1)]['LatestQtr'].value_counts())
  print('-------')

  # criteria of portfolio stocks
  print('Portfolio - Criteria:')
  print(df_mypf[(df_mypf['InPortfolio'] == 1)]['Criteria'].value_counts())
  print('-------')

  # strategies in portfolio stocks
  print('Portfolio - Strategy:')
  print(df_mypf[(df_mypf['InPortfolio'] == 1)]['Strategy'].value_counts())
  print('-------')

  # conviction distribution
  print('Portfolio - Conviction:')
  print(df_mypf[(df_mypf['InPortfolio'] == 1)]['Conviction'].value_counts())

print_distribution()

Overall - Star Stocks:
1.0    95
0.0    91
Name: StarStock, dtype: int64
-------
Overall - V40/V40N Stocks:
V40N    39
V40     38
VR      23
V200     6
AR       4
SS       1
Name: Criteria, dtype: int64
-------
Overall - Latest Qtr:
0    96
1    91
Name: LatestQtr, dtype: int64
-------
Overall - Star + V40/V40N Stocks:
1.0    43
0.0    34
Name: StarStock, dtype: int64
-------
Portfolio - Stocks:
0    135
1     52
Name: InPortfolio, dtype: int64
-------
Portfolio - Star Stocks:
1.0    31
0.0    21
Name: StarStock, dtype: int64
-------
Portfolio - Latest Quarter:
0    28
1    24
Name: LatestQtr, dtype: int64
-------
Portfolio - Criteria:
VR      23
V40N    11
V40     11
AR       4
V200     3
Name: Criteria, dtype: int64
-------
Portfolio - Strategy:
ATH     36
2T3Y    14
BTT      2
Name: Strategy, dtype: int64
-------
Portfolio - Conviction:
M    27
L    14
H    11
Name: Conviction, dtype: int64


In [7]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

In [8]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_features(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  stocks = ['INDIGOPNTS', 'IRCTC', 'GILLETTE', 'PAGEIND', 'BANDHANBNK', 'ABBOTINDIA', 'JCHAC',
              'NESTLEIND', 'PGHH', 'EQUITASBNK', 'SBICARD', 'ASTRAZEN', 'ICICIGI', 'RELAXO']

  if stock_name in stocks:
    actual_link = link_blueprint + stock_name.upper()
  else:
    actual_link = link_blueprint + stock_name.upper() + "/consolidated/"

  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:

    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
    current_roce = float(test_list[test_list.index('ROCE')+1:test_list.index('ROCE')+2][0])
    current_roe = float(test_list[test_list.index('ROE')+1:test_list.index('ROE')+2][0])
  except ValueError as ve:
    current_pe = 1000
    current_roce = 1000
    current_roe = 1000

  return current_pe, current_roce, current_roe

In [9]:
# function to compute stock attributes
def get_common_features():
  def stock_prec_dev(stock_symbol):
      #stock_symbol = 'ULTRACEMCO.NS'
      short_window = 20
      mid_window = 50
      long_window = 200
      moving_avg = 'SMA'

      # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
      start = datetime.datetime(*map(int, start_date.split('-')))
      end = datetime.datetime(*map(int, end_date.split('-')))
      stock_df = yfin.Ticker(stock_symbol).history(period='10y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
      stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
      stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
      stock_df.dropna(axis = 0, inplace = True) # remove any null rows

      stock_df['Stock'] = stock_symbol

      stock_df['Close'] = round(stock_df['Close'],2)
      stock_df['Max'] = round(max(stock_df['Close']),0)
      stock_df.ta.rsi(append=True)
      #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
      stock_df['Close'] = round(stock_df['Close'],0)
      stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
      stock_df.drop(['Open', 'Low', 'High', 'Volume'
                    ], axis=1, inplace=True)

      return stock_df

  df_prec_dev = pd.DataFrame()

  #tmp = stock_prec_dev('ULTRACEMCO.NS')
  #df_perform = df_perform.append(tmp)
  for stock_id in stock_ids:
      #print(stock_id)
      current_pe, current_roce, current_roe = get_current_features(stock_id)
      stock_id = stock_id.upper() + '.NS'
      tmp = stock_prec_dev(stock_id)
      tmp['Prev_Close'] = tmp['Close'].shift(1)
      tmp = tmp.reset_index()
      tmp = tmp.tail(1)
      tmp['Avg_Cost'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
      tmp['Shares'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]
      tmp['Med_PE'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['MedianPE'].values[0]
      tmp['Curr_PE'] = current_pe
      tmp['Curr_ROCE%'] = current_roce
      tmp['Curr_ROE%'] = current_roe
      tmp['Criteria'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Criteria'].values[0]
      tmp['Strategy'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Strategy'].values[0]
      tmp['Target'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Target'].values[0]
      tmp['LatestQtr'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['LatestQtr'].values[0]
      tmp['StarStock'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['StarStock'].values[0]
      tmp['Conviction'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Conviction'].values[0]
      tmp['Investment'] = tmp['Avg_Cost'] * tmp['Shares']
      df_prec_dev = df_prec_dev.append(tmp)
  return df_prec_dev, stock_prec_dev

df_common_features, stock_prec_dev = get_common_features()

In [10]:
# nifty in last 5 weeks
stock_prec_dev('^NSEI')[-21::5]

,Close,200_SMA,Dev%_200,Stock,Max,RSI_14
Date,,,,,,
2023-07-06 00:00:00+05:30,19497.0,17983.0,8.42,^NSEI,19979.0,78.0
2023-07-13 00:00:00+05:30,19414.0,18025.0,7.70,^NSEI,19979.0,69.0
2023-07-20 00:00:00+05:30,19979.0,18090.0,10.44,^NSEI,19979.0,81.0
2023-07-27 00:00:00+05:30,19660.0,18156.0,8.28,^NSEI,19979.0,61.0
2023-08-03 00:00:00+05:30,19473.0,18218.0,6.89,^NSEI,19979.0,50.0


In [11]:
stock_ids = df_common_features['Stock'].values
df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)

#df_common_features['Stock'] = df_common_features['Symbol'] + '.NS'
#df_prec_dev.drop(['Med_PE'], axis=1, inplace=True)

In [12]:
df_common_features['Discount%PE'] = round((df_common_features['Med_PE']-df_common_features['Curr_PE'])*100/(df_common_features['Med_PE']),0)
df_common_features['Target'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
df_common_features['Upside%Target'] = round((df_common_features['Target']-df_common_features['Close'])*100/(df_common_features['Close']),0)
df_common_features = df_common_features.set_index('Stock')

In [13]:
# portfolio stocks
cols = ['Close',	'Dev%_200',	'Upside%Target',	'RSI_14', 'Curr_ROCE%', 'Curr_ROE%', 'Conviction',  'LatestQtr', 'StarStock', 'Criteria', 'Strategy', 'Investment']
tmp_df = df_common_features[cols]
tmp_df = tmp_df[tmp_df.index.isin(df_mypf[df_mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))

investment = round(sum(tmp_df['Investment']),0)
tmp_df['Weightage%'] = round(tmp_df['Investment']*100/investment,2)
tmp_df.sort_values(by = 'Upside%Target', ascending=False)

qualified stocks: 52


,Close,Dev%_200,Upside%Target,RSI_14,Curr_ROCE%,Curr_ROE%,Conviction,LatestQtr,StarStock,Criteria,Strategy,Investment,Weightage%
Stock,,,,,,,,,,,,,
PNB.NS,60.0,16.76,255.0,51.0,4.11,3.34,M,1,1.0,VR,ATH,76958.00,2.22
VALIANTORG.NS,538.0,-0.10,244.0,56.0,14.80,13.00,L,0,0.0,VR,2T3Y,98532.85,2.84
GLAND.NS,1285.0,-7.15,236.0,69.0,14.80,10.90,L,0,0.0,VR,2T3Y,67422.85,1.95
BANDHANBNK.NS,226.0,-3.63,220.0,50.0,6.67,11.90,M,1,0.0,VR,2T3Y,97808.00,2.82
NIACL.NS,123.0,10.95,195.0,58.0,4.86,4.06,L,0,0.0,VR,ATH,45657.30,1.32
LUXIND.NS,1586.0,5.69,187.0,61.0,12.70,9.95,M,0,0.0,V40N,2T3Y,76179.72,2.20
VAIBHAVGBL.NS,358.0,13.31,170.0,74.0,11.10,9.02,L,0,0.0,VR,2T3Y,45717.12,1.32
J&KBANK.NS,67.0,26.89,167.0,51.0,4.92,13.10,M,1,1.0,VR,ATH,30068.75,0.87
METROPOLIS.NS,1375.0,0.71,150.0,43.0,17.40,15.20,L,0,0.0,VR,ATH,30051.56,0.87


In [14]:
# top 5 from portfolio to book profit based on Upside%ATH
tmp_df.sort_values(by = 'Upside%Target', ascending=True).head(7)

,Close,Dev%_200,Upside%Target,RSI_14,Curr_ROCE%,Curr_ROE%,Conviction,LatestQtr,StarStock,Criteria,Strategy,Investment,Weightage%
Stock,,,,,,,,,,,,,
PIDILITIND.NS,2608.0,3.78,11.0,48.0,24.6,18.7,H,0,1.0,V40,ATH,75121.80,2.17
TCS.NS,3424.0,5.41,12.0,57.0,59.1,46.9,H,1,1.0,V40,ATH,51696.00,1.49
EICHERMOT.NS,3317.0,-0.84,16.0,44.0,27.4,21.1,H,0,1.0,V40N,ATH,48903.00,1.41
UJJIVANSFB.NS,48.0,61.00,19.0,72.0,1000.0,1000.0,M,1,1.0,V40N,ATH,68265.00,1.97
ICICIGI.NS,1354.0,15.00,19.0,53.0,21.2,17.6,H,1,1.0,V40,ATH,98916.72,2.85
5PAISA.NS,445.0,36.18,24.0,62.0,12.4,10.6,M,1,1.0,V40N,ATH,79129.68,2.28
BATAINDIA.NS,1744.0,10.35,24.0,72.0,19.6,19.8,H,0,0.0,V40,ATH,152362.00,4.40


In [15]:
# top 10 stocks from portfolio for SIP based on 200 DMA and RSI
tmp_df[tmp_df['Dev%_200'] < 0].sort_values(by = 'RSI_14', ascending=True).head(10)

,Close,Dev%_200,Upside%Target,RSI_14,Curr_ROCE%,Curr_ROE%,Conviction,LatestQtr,StarStock,Criteria,Strategy,Investment,Weightage%
Stock,,,,,,,,,,,,,
FINEORG.NS,4510.0,-11.53,60.0,36.0,65.30,49.4,M,1,0.0,V40N,ATH,51671.73,1.49
KOTAKBANK.NS,1817.0,-1.51,25.0,38.0,6.86,14.2,L,1,1.0,V40,BTT,50854.70,1.47
TEAMLEASE.NS,2305.0,-4.98,129.0,40.0,14.80,15.1,M,1,1.0,V40N,2T3Y,47878.53,1.38
RAJESHEXPO.NS,512.0,-22.37,85.0,41.0,10.30,10.5,M,0,1.0,V40N,ATH,72182.50,2.08
DMART.NS,3684.0,-1.18,45.0,44.0,20.10,16.0,M,1,1.0,AR,ATH,51232.44,1.48
EICHERMOT.NS,3317.0,-0.84,16.0,44.0,27.40,21.1,H,0,1.0,V40N,ATH,48903.00,1.41
CLEAN.NS,1314.0,-8.51,101.0,45.0,44.50,33.2,L,0,1.0,VR,2T3Y,53239.29,1.54
BANDHANBNK.NS,226.0,-3.63,220.0,50.0,6.67,11.9,M,1,0.0,VR,2T3Y,97808.00,2.82
INFY.NS,1352.0,-3.91,37.0,50.0,40.70,31.8,H,1,1.0,V40,ATH,137703.28,3.97


In [16]:
#df_common_features.drop([ '200_SMA', 'Avg_Cost', 'Shares', 'Med_PE', 'Curr_PE', 'Discount%PE'], axis=1, inplace=True)
tmp_base_df = df_common_features[~df_common_features.index.isin(df_mypf[df_mypf['InPortfolio'] == 1].Stock.values)][cols]
tmp_base_df.drop(['Investment'], axis=1, inplace=True)

In [17]:
# latest quarter prospects (for upside)

tmp_df = tmp_base_df[tmp_base_df.index.isin(my_prospects)]
#tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
#tmp_df = tmp_df[(tmp_df['Upside%Target'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=False)

qualified stocks: 0


,Close,Dev%_200,Upside%Target,RSI_14,Curr_ROCE%,Curr_ROE%,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,


In [18]:
# star + v40/v40n prospects (for upside)

tmp_df = tmp_base_df[tmp_base_df.index.isin(star_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%Target'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=False)

qualified stocks: 7


,Close,Dev%_200,Upside%Target,RSI_14,Curr_ROCE%,Curr_ROE%,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,
MOTILALOFS.NS,810.0,21.47,82.0,63.0,13.0,15.6,L,1,1.0,V40N,NaN
DIXON.NS,4106.0,12.50,40.0,45.0,25.8,22.4,M,1,1.0,V40N,NaN
ANGELONE.NS,1479.0,9.37,27.0,38.0,44.0,47.1,M,1,1.0,V40N,NaN
BAJFINANCE.NS,7203.0,9.93,27.0,42.0,11.8,23.5,H,1,1.0,V40,BTT
ASIANPAINT.NS,3363.0,10.69,26.0,48.0,35.2,27.7,H,1,1.0,V40,BTT
ASTRAZEN.NS,3806.0,12.45,22.0,57.0,30.9,23.0,M,0,1.0,V40N,NaN
BERGEPAINT.NS,712.0,16.66,21.0,72.0,23.8,20.4,M,0,1.0,V40,NaN


In [19]:
# star + v40/v40n prospects (for breakout)

tmp_df = tmp_base_df[tmp_base_df.index.isin(star_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%Target'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=False)

qualified stocks: 8


,Close,Dev%_200,Upside%Target,RSI_14,Curr_ROCE%,Curr_ROE%,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,
AXISBANK.NS,945.0,4.79,4.0,43.0,6.16,14.5,M,1,1.0,V40,NaN
CAPLIPOINT.NS,909.0,25.57,4.0,70.0,26.80,22.4,M,0,1.0,V40N,NaN
CERA.NS,7739.0,23.13,4.0,56.0,25.80,19.4,H,0,1.0,V40N,NaN
MCDOWELL-N.NS,998.0,17.74,4.0,60.0,19.90,17.5,L,1,1.0,V40N,NaN
3MINDIA.NS,28267.0,19.01,3.0,57.0,31.40,23.4,M,0,1.0,V40N,NaN
BAJAJ-AUTO.NS,4832.0,24.60,2.0,55.0,26.20,20.2,L,1,1.0,V40,NaN
ICICIBANK.NS,980.0,8.07,2.0,55.0,6.82,18.7,H,1,1.0,V40,NaN
MARICO.NS,567.0,11.26,1.0,63.0,43.00,35.8,M,1,1.0,V40,NaN


In [20]:
# star stocks (for upside)

tmp_df = tmp_base_df[tmp_base_df.index.isin(star_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] != 'V40') & (tmp_df['Criteria'] != 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%Target'] >= 25)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=False)

qualified stocks: 7


,Close,Dev%_200,Upside%Target,RSI_14,Curr_ROCE%,Curr_ROE%,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,
ADANITRANS.NS,789.0,-51.37,420.0,49.0,10.50,11.60,L,1,1.0,NaN,NaN
ADANIGREEN.NS,1052.0,-19.29,182.0,58.0,7.81,22.80,L,1,1.0,NaN,NaN
IEX.NS,128.0,-8.06,124.0,57.0,53.00,40.70,M,1,1.0,NaN,ATH
ADANIENT.NS,2474.0,-7.77,68.0,59.0,10.10,9.63,M,0,1.0,NaN,NaN
GODREJIND.NS,476.0,6.19,43.0,43.0,9.75,11.20,L,0,1.0,NaN,NaN
CANBK.NS,327.0,10.34,30.0,48.0,5.33,14.80,H,1,1.0,NaN,NaN
SBICARD.NS,869.0,7.71,30.0,54.0,13.00,25.70,M,1,1.0,NaN,NaN


In [21]:
# star stocks (for breakout)

tmp_df = tmp_base_df[tmp_base_df.index.isin(star_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%Target'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
#print(tmp_df.index)
tmp_df.sort_values(by = 'Upside%Target', ascending=False)

qualified stocks: 8


,Close,Dev%_200,Upside%Target,RSI_14,Curr_ROCE%,Curr_ROE%,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,
AXISBANK.NS,945.0,4.79,4.0,43.0,6.16,14.5,M,1,1.0,V40,NaN
CAPLIPOINT.NS,909.0,25.57,4.0,70.0,26.80,22.4,M,0,1.0,V40N,NaN
CERA.NS,7739.0,23.13,4.0,56.0,25.80,19.4,H,0,1.0,V40N,NaN
MCDOWELL-N.NS,998.0,17.74,4.0,60.0,19.90,17.5,L,1,1.0,V40N,NaN
3MINDIA.NS,28267.0,19.01,3.0,57.0,31.40,23.4,M,0,1.0,V40N,NaN
BAJAJ-AUTO.NS,4832.0,24.60,2.0,55.0,26.20,20.2,L,1,1.0,V40,NaN
ICICIBANK.NS,980.0,8.07,2.0,55.0,6.82,18.7,H,1,1.0,V40,NaN
MARICO.NS,567.0,11.26,1.0,63.0,43.00,35.8,M,1,1.0,V40,NaN


In [22]:
# stocks excluded from all time high

excluded_stocks = ['ACC.NS','AMBUJACEM.NS','CADILAHC.NS','GILLETTE.NS','NESTLEIND.NS','PGHH.NS', 'PVRINOX.NS', 'SIEMENS.NS']

tmp_df = tmp_base_df[tmp_base_df.index.isin(excluded_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=False)

qualified stocks: 2


,Close,Dev%_200,Upside%Target,RSI_14,Curr_ROCE%,Curr_ROE%,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,
NESTLEIND.NS,22705.0,10.93,2.0,50.0,138.0,108.0,H,0,0.0,V40,NaN
PGHH.NS,15700.0,11.85,2.0,60.0,110.0,79.5,L,0,0.0,V40,NaN


In [23]:
# stocks prospects

excluded_stocks = ['BANDHANBNK.NS','GICRE.NS','JMFINANCIL.NS','NIACL.NS','PNB.NS','UJJIVANSFB.NS']

tmp_df = tmp_base_df[tmp_base_df.index.isin(excluded_stocks)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=False)

qualified stocks: 0


,Close,Dev%_200,Upside%Target,RSI_14,Curr_ROCE%,Curr_ROE%,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,


In [24]:
# prospects with heavy upside
tmp_df = tmp_base_df[(tmp_base_df['Criteria'] == 'V40') | (tmp_base_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[tmp_df['Upside%Target'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=False)

qualified stocks: 10


,Close,Dev%_200,Upside%Target,RSI_14,Curr_ROCE%,Curr_ROE%,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,
JCHAC.NS,957.0,-14.25,236.0,32.0,1000.00,1000.00,M,1,0.0,V40N,NaN
SYMPHONY.NS,874.0,-7.14,136.0,46.0,14.80,13.50,M,1,0.0,V40N,NaN
WHIRLPOOL.NS,1447.0,1.63,84.0,51.0,8.87,6.37,M,0,0.0,V40,NaN
LALPATHLAB.NS,2303.0,6.77,82.0,50.0,18.50,15.00,M,1,0.0,V40N,NaN
MOTILALOFS.NS,810.0,21.47,82.0,63.0,13.00,15.60,L,1,1.0,V40N,NaN
SFL.NS,1170.0,-1.64,69.0,48.0,13.60,12.50,M,0,0.0,V40N,NaN
SIS.NS,433.0,11.41,57.0,56.0,10.60,15.60,M,1,0.0,V40N,NaN
PFIZER.NS,4018.0,0.40,50.0,73.0,27.20,19.80,L,0,0.0,V40,NaN
TTKPRESTIG.NS,782.0,0.90,48.0,60.0,18.20,13.80,M,0,0.0,V40N,NaN


In [25]:
# cross-check the features
tmp_df = tmp_base_df[(tmp_base_df['Criteria'] == 'V40') | (tmp_base_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[tmp_df['Upside%Target'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=False)

qualified stocks: 10


,Close,Dev%_200,Upside%Target,RSI_14,Curr_ROCE%,Curr_ROE%,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,
JCHAC.NS,957.0,-14.25,236.0,32.0,1000.00,1000.00,M,1,0.0,V40N,NaN
SYMPHONY.NS,874.0,-7.14,136.0,46.0,14.80,13.50,M,1,0.0,V40N,NaN
WHIRLPOOL.NS,1447.0,1.63,84.0,51.0,8.87,6.37,M,0,0.0,V40,NaN
LALPATHLAB.NS,2303.0,6.77,82.0,50.0,18.50,15.00,M,1,0.0,V40N,NaN
MOTILALOFS.NS,810.0,21.47,82.0,63.0,13.00,15.60,L,1,1.0,V40N,NaN
SFL.NS,1170.0,-1.64,69.0,48.0,13.60,12.50,M,0,0.0,V40N,NaN
SIS.NS,433.0,11.41,57.0,56.0,10.60,15.60,M,1,0.0,V40N,NaN
PFIZER.NS,4018.0,0.40,50.0,73.0,27.20,19.80,L,0,0.0,V40,NaN
TTKPRESTIG.NS,782.0,0.90,48.0,60.0,18.20,13.80,M,0,0.0,V40N,NaN


In [26]:
# analyse individual stock
stock_id = 'PVRINOX.NS'
df_common_features[df_common_features.index == stock_id]

,Close,200_SMA,Dev%_200,Max,RSI_14,Prev_Close,Avg_Cost,Shares,Med_PE,Curr_PE,...,Curr_ROE%,Criteria,Strategy,Target,LatestQtr,StarStock,Conviction,Investment,Discount%PE,Upside%Target
Stock,,,,,,,,,,,,,,,,,,,,,
PVRINOX.NS,1621.0,1595.0,1.62,1917.0,77.0,1529.0,NaN,NaN,51.2,1000.0,...,1000.0,NaN,NaN,1917.0,0,0.0,L,NaN,-1853.0,18.0


In [27]:
# save the output
if gen_output == 1:
  from google.colab import drive
  drive.mount('/content/drive')
  df_prec_dev = df_prec_dev.reset_index()
  df_prec_dev.to_csv('/content/drive/My Drive/data/stocks/myProspects-Upside-Analysis-ATH.csv', index=False)

In [28]:
# stock in last 5 weeks
#stock_ids = ['ICICIGI','DABUR','MOTILALOFS','JPPOWER','EQUITAS','GLAND','JMFINANCIL','LUXIND','UJJIVANSFB','VAIBHAVGBL','VALIANTORG']
stock_prec_dev('BERGEPAINT.NS')[-21::5]

,Close,200_SMA,Dev%_200,Stock,Max,RSI_14
Date,,,,,,
2023-07-06 00:00:00+05:30,681.0,605.0,12.50,BERGEPAINT.NS,858.0,65.0
2023-07-13 00:00:00+05:30,662.0,606.0,9.16,BERGEPAINT.NS,858.0,46.0
2023-07-20 00:00:00+05:30,684.0,607.0,12.77,BERGEPAINT.NS,858.0,63.0
2023-07-27 00:00:00+05:30,674.0,608.0,10.86,BERGEPAINT.NS,858.0,51.0
2023-08-03 00:00:00+05:30,713.0,610.0,16.93,BERGEPAINT.NS,858.0,72.0
